# Using Schema Registry and Protobuf

In [ ]:
# import json
# import uuid
# import os
# from dotenv import load_dotenv
# from pathlib import Path
# from faker import Faker
# from time import sleep
# from confluent_kafka import SerializingProducer
# from confluent_kafka.serialization import StringSerializer
# import random
# from confluent_kafka.schema_registry import SchemaRegistryClient
# import sys
# from confluent_kafka.schema_registry.protobuf import ProtobufSerializer

# # Load environment variables from .env file
# dotenv_path = Path('/opt/app/.env')
# load_dotenv(dotenv_path=dotenv_path)

# kafka_host = os.getenv('KAFKA_HOST')
# topic_name = os.getenv('KAFKA_TOPIC_NAME')

# # Protobuf schema import
# sys.path.append('/home/jovyan/protobuf')
# import schema_pb2

# # Kafka Configuration
# bootstrap_servers = f'{kafka_host}:9092'

# # Schema Registry Client
# schema_registry_client = SchemaRegistryClient({
#     'url': f'http://{os.getenv("SCHEMA_REG_HOST")}:8081'
# })

# # Protobuf Serializer
# protobuf_serializer = ProtobufSerializer(
#     schema_pb2.PetShopEvent, 
#     schema_registry_client, 
#     conf={'use.deprecated.format': False} 
# )

# # Producer Configuration (using protobuf_serializer for value)
# producer_conf = {
#     'bootstrap.servers': bootstrap_servers,
#     'key.serializer': StringSerializer('utf_8'),  
#     'value.serializer': protobuf_serializer,  
#     'message.timeout.ms': 10000
# }

# # Initialize Kafka producer
# producer = SerializingProducer(producer_conf)

# # Initialize Faker for fake data generation
# fake = Faker()

# # Generate a fake PetShopEvent using Faker and Protobuf
# def generate_pet_shop_event():
#     pet_event = schema_pb2.PetShopEvent()
#     pet_event.pet_id = str(uuid.uuid4())
#     pet_event.pet_name = fake.first_name()
#     pet_event.species = fake.random_element(elements=('Dog', 'Cat', 'Bird', 'Reptile'))
#     pet_event.breed = fake.random_element(elements=('Bulldog', 'Siamese', 'Parrot', 'Iguana'))
#     pet_event.age = fake.random_int(min=1, max=20)
#     pet_event.price = round(random.uniform(50.00, 5000.00), 2)
#     pet_event.weight = round(random.uniform(1.0, 100.0), 2)
#     pet_event.checkup_date = fake.date_this_decade().strftime('%Y-%m-%d')  # Convert date to string
#     return pet_event

# # Generate and send one event every 5 seconds
# for i in range(1, 400):
#     pet_event = generate_pet_shop_event()
    
#     # Print the generated fake PetShopEvent (Protobuf object)
#     print(f"Generated PetShopEvent: {pet_event}")
    
#     # Send the event to Kafka topic
#     producer.produce(topic=topic_name, key=pet_event.pet_id, value=pet_event)
    
#     # Flush to ensure the message is sent
#     producer.flush

#     # Wait before sending the next event
#     sleep(5)

# Without Schema Registry and Protobuf

In [ ]:
import json
import uuid
import os
from dotenv import load_dotenv
from pathlib import Path
from faker import Faker
from time import sleep
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import StringSerializer
import random

# Load environment variables from .env file
dotenv_path = Path('/opt/app/.env')
load_dotenv(dotenv_path=dotenv_path)

kafka_host = os.getenv('KAFKA_HOST')
topic_name = os.getenv('KAFKA_TOPIC_NAME')

# Kafka Configuration
bootstrap_servers = f'{kafka_host}:9092'

# Producer Configuration (using JSON for value)
producer_conf = {
    'bootstrap.servers': bootstrap_servers,
    'key.serializer': StringSerializer('utf_8'),
    'value.serializer': StringSerializer('utf_8'),
    'message.timeout.ms': 10000
}

# Initialize Kafka producer
producer = SerializingProducer(producer_conf)

# Initialize Faker for fake data generation
fake = Faker()

# Generate a fake PetShopEvent using Faker and JSON
def generate_pet_shop_event():
    pet_event = {
        "pet_id": str(uuid.uuid4()),
        "pet_name": fake.first_name(),
        "species": fake.random_element(elements=('Dog', 'Cat', 'Bird', 'Reptile')),
        "breed": fake.random_element(elements=('Bulldog', 'Siamese', 'Parrot', 'Iguana')),
        "age": fake.random_int(min=1, max=20),
        "price": round(random.uniform(50.00, 5000.00), 2),
        "weight": round(random.uniform(1.0, 100.0), 2),
        "checkup_date": fake.date_this_decade().strftime('%Y-%m-%d')  # Convert date to string
    }
    return json.dumps(pet_event)  # Convert the event to a JSON string

# Generate and send one event every 5 seconds
for i in range(1, 400):
    pet_event = generate_pet_shop_event()
    
    # Print the generated fake PetShopEvent (JSON object)
    print(f"Generated PetShopEvent: {pet_event}")
    
    # Send the event to Kafka topic
    producer.produce(topic=topic_name, key=str(uuid.uuid4()), value=pet_event)
    
    # Flush to ensure the message is sent
    producer.flush()

    # Wait before sending the next event
    sleep(5)
